<a href="https://colab.research.google.com/github/liwenjing2012/Cousera_Capstone/blob/main/Capstone_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cousera Capstone Project : Neighborhoods Battle
Where is

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [ ]:
import urllib.request

In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [ ]:
CLIENT_ID = "KU0CC2XGBFTS40SCQZYT5C1W4JMNIRA1D5H3SVZCITNDO3XF"
CLIENT_SECRET="HCOL1SIL4HGTJKTXPG34GVCF0YIPNY4PIOTVLDTERRFQRZYD"
VERSION = '20180604'
ACCESS_TOKEN = '0GFYVQWD4RZ2YPEWEFDYCKM0FN5ADDL1KVYPC1PWASSNHZBH'

latitude = 40.715337
longitude = -74.008848
radius = 500
LIMIT = 100
search_query = 'housing'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
# send GET request and get trending venues
results = requests.get(url).json()
results

<Response [200]>


In [ ]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4bbfd504461576b057547932,New York City Housing Court,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",v-1623413012,False,111 Centre St,btwn Leonard & White,40.716811,-74.002015,"[{'label': 'display', 'lat': 40.71681123524968...",599,10013,US,New York,NY,United States,"[111 Centre St (btwn Leonard & White), New Yor..."
1,4e8c9ecc4901755a2e292434,Housing conference room,"[{'id': '4bf58dd8d48988d100941735', 'name': 'M...",v-1623413012,False,NaN,NaN,40.714114,-74.009358,"[{'label': 'display', 'lat': 40.71411438661994...",142,10007,US,Manhattan,NY,United States,"[Manhattan, NY 10007]"
2,51aa9f36498e6e87c2f3c43e,"housing works chambers st, nyc","[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1623413012,False,NaN,NaN,40.714169,-74.006472,"[{'label': 'display', 'lat': 40.71416887391371...",238,NaN,US,New York,NY,United States,"[New York, NY]"
3,509819dbe4b0e52fa55ebd47,New York City Housing Authority,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1623413012,False,90 Church Street,NaN,40.711893,-74.011019,"[{'label': 'display', 'lat': 40.71189329396107...",424,NY,US,New York,NY,United States,"[90 Church Street, New York, NY NY]"
4,4e4abe4afa76a0c058d425ca,Housing Resources Administration,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1623413012,False,NaN,NaN,40.718037,-74.005707,"[{'label': 'display', 'lat': 40.71803665161133...",400,NaN,US,New York,NY,United States,"[New York, NY]"


In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pisillo Italian Panini,Sandwich Place,97 Nassau St,at Ann St,40.710530,-74.007526,"[{'label': 'display', 'lat': 40.71053008446078...",546,10038,US,New York,NY,United States,"[97 Nassau St (at Ann St), New York, NY 10038]",528bf16711d2b7722da6b51c
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,NaN,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",124,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007]",4f3232e219836c91c7bfde94
2,Caruso's Italian Restaurant,Italian Restaurant,42 Broadway,NaN,40.710509,-74.008276,"[{'label': 'display', 'lat': 40.71050909999999...",539,10004,US,New York,NY,United States,"[42 Broadway, New York, NY 10004]",4e4c4d28bd413c4cc66857a5
3,Pisillo Italian Cafe,Café,99 Nassau St,NaN,40.710458,-74.007693,"[{'label': 'display', 'lat': 40.710458, 'lng':...",551,10038,US,New York,NY,United States,"[99 Nassau St, New York, NY 10038]",6065ecdcc85e724bffef8c08
4,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",497,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282]",4fa862b3e4b0ebff2f749f06
5,Gloria's Restaurant,Italian Restaurant,107 W Broadway,Reade St,40.715794,-74.008789,"[{'label': 'display', 'lat': 40.71579360961914...",51,10013,US,New York,NY,United States,"[107 W Broadway (Reade St), New York, NY 10013]",4c5de89e6ebe2d7f7955d42e


In [ ]:
url = "https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Crime_Reports.csv"
criminal_data = pd.read_csv(url)
criminal_data.head()


,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,Council District,APD Sector,APD District,PRA,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,20201530258,ROBBERY BY ASSAULT,303,N,06/01/2020 03:30:00 AM,06/01/2020,330,06/01/2020 03:58:00 AM,06/01/2020,358,BAR / NIGHT CLUB,9621 N IH 35 SVRD NB,78753.0,4.0,ED,7,241.0,18.33,N,06/18/2020,120,Robbery,3131655.0,3131655.0,30.361035,-97.685308,"(30.36103465, -97.68530777)"
1,20201530887,FAMILY DISTURBANCE,3400,N,06/01/2020 02:02:00 PM,06/01/2020,1402,06/01/2020 02:02:00 PM,06/01/2020,1402,RESIDENCE / HOME,2302 SWEET CLOVER DR,78745.0,5.0,FR,1,525.0,17.28,NaN,NaN,NaN,NaN,3090537.0,3090537.0,30.190364,-97.820164,"(30.19036406, -97.82016354)"
2,20201530201,BURGLARY NON RESIDENCE,502,N,06/01/2020 03:22:00 AM,06/01/2020,322,06/01/2020 03:22:00 AM,06/01/2020,322,DRUG STORE / DR. OFFICE / HOSPITAL,1909 1/2 E 38TH ST,78723.0,1.0,BA,4,900.0,3.00,N,06/03/2020,220,Burglary,3125696.0,3125696.0,30.287689,-97.706205,"(30.28768858, -97.70620522)"
3,20205021293,BURGLARY OF VEHICLE,601,N,06/01/2020 07:50:00 AM,06/01/2020,750,06/01/2020 10:11:00 AM,06/01/2020,1011,RESIDENCE / HOME,408 KREBS LN,78704.0,3.0,DA,2,474.0,NaN,N,06/02/2020,23F,Theft,3107193.0,3107193.0,30.227088,-97.766456,"(30.22708779, -97.76645599)"
4,20205021334,BURGLARY OF VEHICLE,601,N,05/30/2020 11:00:00 PM,05/30/2020,2300,06/01/2020 12:22:00 PM,06/01/2020,1222,PARKING LOTS / GARAGE,3105 N LAMAR BLVD,78705.0,9.0,BA,3,345.0,2.00,N,06/02/2020,23F,Theft,3113056.0,3113056.0,30.300634,-97.745917,"(30.30063372, -97.74591661)"


In [ ]:
criminal_features = ['Highest Offense Description', 'Category Description','Family Violence','Occurred Date', 'Location Type', 'Zip Code', 'Latitude', 'Longitude']
criminal_feature_data = criminal_data[criminal_features]

criminal_feature_data.groupby(['Family Violence','Highest Offense Description']).count()
#criminal_feature_data[criminal_feature_data['Category Description'].isna()].reset_index(drop=True).groupby(['Family Violence']).count()

Category Description  ...  Longitude
Family Violence Highest Offense Description                           ...           
N               ABUSE OF 911                                       0  ...         36
                AGG ASLT STRANGLE/SUFFOCATE                       11  ...         10
                AGG ASSAULT                                     1164  ...       1156
                AGG ASSAULT FAM/DATE VIOLENCE                     10  ...         10
                AGG ASSAULT ON PEACE OFFICER                      25  ...         25
...                                                              ...  ...        ...
Y               THEFT FROM PERSON                                 16  ...         15
                THEFT OF BICYCLE                                   1  ...          1
                UNLAWFUL RESTRAINT                                 0  ...          8
                UNLAWFUL RESTRAINT FAM/DAT VIO                     0  ...         25
                VIOL OF EMERG PROTECTIVE ORDER                     0  ...          1

[386 rows x 6 columns]

In [ ]:
!pip install rtree
!pip install geopandas

     |████████████████████████████████| 1.0MB 2.9MB/s 


In [ ]:
from shapely.geometry import Point
import geopandas as gpd




In [ ]:
url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Neighborhoods.geojson'
geoNeigh = gpd.read_file(url)
geoNeigh_data = geoNeigh[['neighname','geometry']]
geoNeigh_data

cirminal_gdf = gpd.GeoDataFrame(
    criminal_feature_data, geometry=gpd.points_from_xy(criminal_feature_data.Longitude, criminal_feature_data.Latitude))

In [ ]:
geoNeigh_data

,neighname,geometry
0,ANDERSON MILL,"MULTIPOLYGON (((-97.79231 30.45671, -97.79154 ..."
1,WINDSOR PARK,"MULTIPOLYGON (((-97.67076 30.30854, -97.67204 ..."
2,DAWSON,"MULTIPOLYGON (((-97.75353 30.23876, -97.75472 ..."
3,WEST UNIVERSITY,"MULTIPOLYGON (((-97.73815 30.30275, -97.73860 ..."
4,MLK,"MULTIPOLYGON (((-97.68262 30.28587, -97.68372 ..."
...,...,...
98,SWEETBRIAR,"MULTIPOLYGON (((-97.77485 30.20490, -97.77444 ..."
99,SPICEWOOD,"MULTIPOLYGON (((-97.77410 30.38888, -97.77410 ..."
100,WINDSOR HILLS,"MULTIPOLYGON (((-97.66564 30.37248, -97.66569 ..."
101,ROGERS HILL,"MULTIPOLYGON (((-97.62863 30.30690, -97.62942 ..."


In [ ]:
cirminal_gdf

,Highest Offense Description,Category Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude,geometry
0,ROBBERY BY ASSAULT,Robbery,N,06/01/2020,BAR / NIGHT CLUB,78753.0,30.361035,-97.685308,POINT (-97.68531 30.36103)
1,FAMILY DISTURBANCE,NaN,N,06/01/2020,RESIDENCE / HOME,78745.0,30.190364,-97.820164,POINT (-97.82016 30.19036)
2,BURGLARY NON RESIDENCE,Burglary,N,06/01/2020,DRUG STORE / DR. OFFICE / HOSPITAL,78723.0,30.287689,-97.706205,POINT (-97.70621 30.28769)
3,BURGLARY OF VEHICLE,Theft,N,06/01/2020,RESIDENCE / HOME,78704.0,30.227088,-97.766456,POINT (-97.76646 30.22709)
4,BURGLARY OF VEHICLE,Theft,N,05/30/2020,PARKING LOTS / GARAGE,78705.0,30.300634,-97.745917,POINT (-97.74592 30.30063)
...,...,...,...,...,...,...,...,...,...
99388,BURGLARY OF VEH-NO SUSPECT/FU,Theft,N,06/01/2021,PARKING LOTS / GARAGE,78741.0,30.223678,-97.730531,POINT (-97.73053 30.22368)
99389,FORGERY BY ALTERATION,NaN,N,06/01/2021,BANKS / SAVINGS & LOAN,78704.0,30.245326,-97.750908,POINT (-97.75091 30.24533)
99390,VIOL OF PROTECTIVE ORDER,NaN,N,06/01/2021,STREETS / HWY / ROAD / ALLEY,78702.0,30.269016,-97.718836,POINT (-97.71884 30.26902)
99391,FRAUD - OTHER,NaN,N,05/04/2021,OTHER / UNKNOWN,78750.0,30.444651,-97.795816,POINT (-97.79582 30.44465)


In [ ]:
cirminal_gdf.set_crs(epsg=4326, inplace=True)
joined_gdf = gpd.sjoin(cirminal_gdf, geoNeigh_data, op='within')

In [ ]:
joined_gdf

,Highest Offense Description,Category Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude,geometry,index_right,neighname
0,ROBBERY BY ASSAULT,Robbery,N,06/01/2020,BAR / NIGHT CLUB,78753.0,30.361035,-97.685308,POINT (-97.68531 30.36103),100,WINDSOR HILLS
76,BURGLARY OF RESIDENCE,Burglary,N,06/01/2020,RESIDENCE / HOME,78753.0,30.357407,-97.673489,POINT (-97.67349 30.35741),100,WINDSOR HILLS
110,DOC DISCHARGE GUN - PUB PLACE,NaN,N,06/01/2020,RESIDENCE / HOME,78753.0,30.357966,-97.677619,POINT (-97.67762 30.35797),100,WINDSOR HILLS
355,FAMILY DISTURBANCE,NaN,N,06/02/2020,RESIDENCE / HOME,78753.0,30.352493,-97.675258,POINT (-97.67526 30.35249),100,WINDSOR HILLS
413,RUNAWAY CHILD,NaN,N,06/02/2020,RESIDENCE / HOME,78753.0,30.355716,-97.680798,POINT (-97.68080 30.35572),100,WINDSOR HILLS
...,...,...,...,...,...,...,...,...,...,...,...
81472,FAMILY DISTURBANCE,NaN,N,03/25/2021,RESIDENCE / HOME,78739.0,30.164562,-97.885228,POINT (-97.88523 30.16456),52,HAYS WARTHA
85568,IDENTITY THEFT,NaN,N,04/07/2021,RESIDENCE / HOME,78739.0,30.169860,-97.889882,POINT (-97.88988 30.16986),52,HAYS WARTHA
85663,IDENTITY THEFT,NaN,N,04/07/2021,RESIDENCE / HOME,78652.0,30.165218,-97.909560,POINT (-97.90956 30.16522),52,HAYS WARTHA
89878,IDENTITY THEFT,NaN,N,04/24/2021,RESIDENCE / HOME,78739.0,30.169857,-97.889550,POINT (-97.88955 30.16986),52,HAYS WARTHA


In [ ]:
joined_gdf.groupby('neighname').count()

,Highest Offense Description,Category Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude,geometry,index_right
neighname,,,,,,,,,,
ALLANDALE,698,357,698,698,698,696,698,698,698,698
ANDERSON MILL,1402,507,1402,1402,1399,1399,1402,1402,1402,1402
AVERY RANCH--LAKELINE,1414,675,1414,1414,1413,1413,1414,1414,1414,1414
BARTON CREEK MALL,428,257,428,428,428,427,428,428,428,428
BARTON HILLS,654,352,654,654,654,653,654,654,654,654
...,...,...,...,...,...,...,...,...,...,...
WINDSOR HILLS,957,369,957,957,957,957,957,957,957,957
WINDSOR PARK,2600,1062,2600,2600,2598,2596,2600,2600,2600,2600
WINDSOR ROAD,226,93,226,226,226,225,226,226,226,226
